<h1 style="text-align:left; font-size:30px; color:#008080; font-weight:bold;">
🎧 Stable Audio — Text → Sound (HF Diffusers)
</h1>

<h2 style="text-align:center;">0 — One-line Goal</h2>

Generate short audio clips from text prompts using Hugging Face’s `StableAudioPipeline` in a clean, reproducible, GPU-friendly workflow.

---

### 🔍 What you will learn
- How to install & load Stable Audio models  
- How to generate sound from text  
- How to control audio length, quality, seed  
- How to save + play audio inside notebook  

---

### 📦 Requirements
You **must** set environment variable:

HUGGINGFACE_TOKEN

with your HF Access Token (read permission).

---


In [6]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Install minimal dependencies (works for Colab)
!pip install --upgrade pip
!pip install diffusers==1.27.0 transformers accelerate safetensors soundfile huggingface-hub==0.18.0
!pip install torch torchaudio --quiet
!pip install torchsde
!pip install diffusers


ERROR: Could not find a version that satisfies the requirement diffusers==1.27.0 (from versions: 0.0.1, 0.0.2, 0.0.3, 0.0.4, 0.1.0, 0.1.1, 0.1.2, 0.1.3, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.2.4, 0.3.0, 0.4.0, 0.4.1, 0.4.2, 0.5.0, 0.5.1, 0.6.0, 0.7.0, 0.7.1, 0.7.2, 0.8.0, 0.8.1, 0.9.0, 0.10.0, 0.10.1, 0.10.2, 0.11.0, 0.11.1, 0.12.0, 0.12.1, 0.13.0, 0.13.1, 0.14.0, 0.15.0, 0.15.1, 0.16.0, 0.16.1, 0.17.0, 0.17.1, 0.18.0, 0.18.1, 0.18.2, 0.19.0, 0.19.1, 0.19.2, 0.19.3, 0.20.0, 0.20.1, 0.20.2, 0.21.0, 0.21.1, 0.21.2, 0.21.3, 0.21.4, 0.22.0, 0.22.1, 0.22.2, 0.22.3, 0.23.0, 0.23.1, 0.24.0, 0.25.0, 0.25.1, 0.26.0, 0.26.1, 0.26.2, 0.26.3, 0.27.0, 0.27.1, 0.27.2, 0.28.0, 0.28.1, 0.28.2, 0.29.0, 0.29.1, 0.29.2, 0.30.0, 0.30.1, 0.30.2, 0.30.3, 0.31.0, 0.32.0, 0.32.1, 0.32.2, 0.33.0, 0.33.1, 0.34.0, 0.35.0, 0.35.1, 0.35.2)
ERROR: No matching distribution found for diffusers==1.27.0


In [9]:
import os
import getpass
from huggingface_hub import login

if "HUGGINGFACE_TOKEN" not in os.environ:
    os.environ["HUGGINGFACE_TOKEN"] = getpass.getpass("🔐 Enter your HuggingFace Token: ")

login(token=os.environ["HUGGINGFACE_TOKEN"])
print("✅ Hugging Face login successful.")


🔐 Enter your HuggingFace Token: ··········
✅ Hugging Face login successful.


In [10]:
import torch
from diffusers import StableAudioPipeline
import soundfile as sf
import json
from datetime import datetime

# GPU fallback
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device ->", device)  # ➤

Device -> cuda


⭐**Load Stable Audio Pipeline**

In [11]:
model_id = "stabilityai/stable-audio-open-1.0"

try:
    pipe = StableAudioPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe = pipe.to(device)
    print(f"✅ Loaded pipeline (FP16, device={device})")  # ➤
except Exception as e:
    print("⚠️ FP16 failed → falling back to CPU/FP32:", e)
    pipe = StableAudioPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
    pipe = pipe.to(device)
    print("✅ Loaded fallback pipeline (FP32)")  # ➤


model_index.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

text_encoder/model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/391 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/624M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


✅ Loaded pipeline (FP16, device=cuda)


In [12]:
# 6 — Helper Functions
def get_sampling_rate(pipe):
    if hasattr(pipe, "vae") and getattr(pipe.vae, "sampling_rate", None):
        return pipe.vae.sampling_rate
    if hasattr(pipe, "feature_extractor") and getattr(pipe.feature_extractor, "sampling_rate", None):
        return pipe.feature_extractor.sampling_rate
    return 44100  # default fallback


def save_audio_and_meta(audio_np, out_path, meta):
    sr = meta.get("sampling_rate", 44100)
    sf.write(out_path, audio_np, sr)

    with open(out_path + ".meta.json", "w") as f:
        json.dump(meta, f, indent=2)

    print(f"🎵 Saved audio → {out_path}")


**7 — Generate Audio**

In [13]:
# ---- PARAMETERS ----
prompt = "A soft meditation bell echoing in a calm temple"
negative_prompt = "harsh noise, distortion"
num_inference_steps = 80
audio_end_in_s = 6.0
num_waveforms_per_prompt = 1
seed = 42

generator = torch.Generator(device).manual_seed(seed)

# ---- GENERATE ----
with torch.autocast(device.type if device.type == "cuda" else "cpu"):
    result = pipe(
        prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_inference_steps,
        audio_end_in_s=audio_end_in_s,
        num_waveforms_per_prompt=num_waveforms_per_prompt,
        generator=generator,
    )

audio_arr = result.audios[0].T.float().cpu().numpy()
sampling_rate = get_sampling_rate(pipe)

meta = {
    "prompt": prompt,
    "negative_prompt": negative_prompt,
    "steps": num_inference_steps,
    "duration_s": audio_end_in_s,
    "seed": seed,
    "sampling_rate": sampling_rate,
    "generated_at": datetime.utcnow().isoformat() + "Z",
}

output_file = "stable_audio_output.wav"
save_audio_and_meta(audio_arr, output_file, meta)  # ➤


  0%|          | 0/80 [00:00<?, ?it/s]

🎵 Saved audio → stable_audio_output.wav


In [14]:
# 8 — Play Audio in Notebook
from IPython.display import Audio, display
display(Audio("stable_audio_output.wav"))  # ➤ inline audio player


<h2 style="text-align:center;">📌 Summary</h2>

- Loaded Stable Audio via Hugging Face  
- Generated short audio from text  
- Saved audio + metadata for reproducibility  
- Played audio directly in notebook  

---

<h2 style="text-align:center;">⚠️ Limitations</h2>

1. Requires GPU for smooth execution  
2. Long audio (15–30s) may cause OOM  
3. FP16 support depends on GPU capability  
4. Quality depends heavily on steps & seed  

---

<h2 style="text-align:center;">🚀 Next Steps</h2>

- Add UI slider for prompt / steps  
- Build a Streamlit web interface  
- Add batch processing for multiple prompts  
